In [4]:
import numpy as np
import cv2
import glob

# Termination criteria for the corner sub-pixel refinement process
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
# Adjust the following parameters according to the chessboard
checkerboard_size = (7, 5)
objp = np.zeros((checkerboard_size[0]*checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []  # 3d points in real-world space
imgpoints = []  # 2d points in image plane

# List of calibration images
images = glob.glob('captured_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, checkerboard_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [51]:
# Load an image to undistort
img_to_undistort = cv2.imread('captured_images/image_18.jpg')
h, w = img_to_undistort.shape[:2]
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort
dst = cv2.undistort(img_to_undistort, mtx, dist, None, new_camera_mtx)

# Crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('undistorted_image.jpg', dst)

True

In [73]:
gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (5, 7), None)

# Focal length in pixels (assuming fx and fy are approximately the same)
focal_length_pixels = (mtx[0, 0] + mtx[1, 1]) / 2  # Average of fx and fy

distance_to_object = 440

if ret:
    # Measure the size of a square
    square_size_pixels = np.linalg.norm(corners[2] - corners[3])
    print(f"Square size in pixels: {square_size_pixels}")

    # Convert pixels to real-world units based on known square size (e.g., 27mm)
   
    square_size_mm = (square_size_pixels / focal_length_pixels) * distance_to_object

    print(f"Square size in mm: {square_size_mm}")

Square size in pixels: 124.19742584228516
Square size in mm: 26.849553228124034


In [74]:
known_square_size_mm = 27  # 
error = abs(square_size_mm - known_square_size_mm)
print(f"Calibration error: {error} mm")

Calibration error: 0.15044677187596633 mm


In [1]:
import cv2
import numpy as np

# Load the chessboard image
img = cv2.imread('captured_images/image_2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Define the chessboard size
chessboard_size = (5, 7)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

# If found, refine the corner positions
if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    
    # Draw and display the corners
    cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
    cv2.imshow('Corners', img)
    cv2.waitKey(5)
    cv2.destroyAllWindows()

In [2]:
p1 = corners2[0].ravel()  # Top-left corner in the image
p2 = corners2[7].ravel()  # Top-right corner in the image
p3 = corners2[-1].ravel()  # Bottom-right corner in the image
p4 = corners2[-8].ravel()  # Bottom-left corner in the image

# 2D image points
img_points = np.array([p1, p2, p3, p4])

# 3D real-world points
obj_points = np.array([[0, 0, 0], [216, 0, 0], [216, 162, 0], [0, 162, 0]], dtype='float32')

In [5]:
ret, rvecs, tvecs = cv2.solvePnP(obj_points, img_points, mtx, dist)

# Convert rotation vectors to rotation matrix
R_mtx, jac = cv2.Rodrigues(rvecs)

# Now, R_mtx is the rotation matrix and tvecs is the translation vector

In [6]:
import math

def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

# Assuming R_mtx is the rotation matrix obtained earlier
euler_angles = rotation_matrix_to_euler_angles(R_mtx)

In [7]:
print("Intrinsic Matrix:")
print(mtx)
print("---------------------------------------------------------------------")
print("Extrinsic Rotation Matrix:")
print(R_mtx)
print("---------------------------------------------------------------------")
print("Extrinsic Translation Vector:")
print(tvecs)
print("---------------------------------------------------------------------")
print("Rotation angles (degrees) around x, y, z axes:")
print(np.degrees(euler_angles))

Intrinsic Matrix:
[[2.02100114e+03 0.00000000e+00 9.28608282e+02]
 [0.00000000e+00 2.04959675e+03 4.87634022e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
---------------------------------------------------------------------
Extrinsic Rotation Matrix:
[[-0.19161793 -0.86009458  0.4727789 ]
 [ 0.37292705  0.38177235  0.84568037]
 [-0.90785901  0.33835956  0.2475981 ]]
---------------------------------------------------------------------
Extrinsic Translation Vector:
[[ 118.65053037]
 [-116.9752828 ]
 [ 577.21351039]]
---------------------------------------------------------------------
Rotation angles (degrees) around x, y, z axes:
[ 53.80481484  65.21113943 117.19507323]


In [9]:
#measuring object dimension

def measure_object_dimension(frame, mtx, dist, distance_to_object):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Blur the image to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection
    edged = cv2.Canny(blur, 35, 125)
    
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours based on their area, keeping only the largest
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    

    for c in contours:
        # Approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # If the approximated contour has four points, assume it's a rectangle (or similar)
        if len(approx) == 4:
            # Draw the contour
            cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
            
            # Compute the bounding box of the contour and use it to compute the object dimensions
            (x, y, w, h) = cv2.boundingRect(approx)
            width_pixels = w  
            height_pixels = h  

        # Calculate real-world dimensions
            width_mm = (width_pixels * distance_to_object) / mtx[0, 0]  # Using f_x from the camera matrix
            height_mm = (height_pixels * distance_to_object) / mtx[1, 1]  # Using f_y from the camera matrix

            # Display the dimensions on the image
            cv2.putText(frame, f"Width: {width_mm:.2f} mm", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(frame, f"Height: {height_mm:.2f} mm", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

In [12]:
#Real time video capturing 

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    
        # Capture frame-by-frame
    ret, frame = cap.read()

        # If frame is read correctly, ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

        # Call the object measurement function on the current frame
    measure_object_dimension(frame, mtx, dist,350)

        # Display the resulting frame
    cv2.imshow('frame', frame)

        # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

    # release the capture
cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 

In [11]:
import numpy as np

# Save to file
np.save('camera_mtx.npy', mtx)
np.save('dist_coeffs.npy', dist)